### VacationPy

In [1]:
#dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
pd.options.mode.chained_assignment = None

#import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [2]:
#import csv
city_df = pd.read_csv("../WeatherPy/output_files/city_data.csv", index_col=0)
city_df.head()

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
0,ushuaia,AR,-54.80,-68.30,50.00,61,20,2.13,1603671119
1,puerto ayora,EC,-0.74,-90.35,68.00,86,53,5.99,1603671267
2,sur,OM,22.57,59.53,72.66,61,0,6.33,1603671267
3,albany,US,42.60,-73.97,43.00,54,1,3.53,1603671267
4,lorengau,PG,-2.02,147.27,86.11,65,100,5.70,1603671268


### Humidity Heatmap

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#create map using lat long as locations
locations = city_df[['Latitude', 'Longitude']].astype(float)
humidity = city_df["Humidity"].astype(float)

#create heatmap figure
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(heat_layer)

#show heatmap
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [5]:
#narrow down df based on conditions
ideal_city = city_df.loc[(city_df["Max Temperature"] > 75)]
ideal_city = ideal_city.loc[(ideal_city["Max Temperature"] < 80)]
ideal_city = ideal_city.loc[(ideal_city["Wind Speed"] < 10)]
ideal_city = ideal_city.loc[(ideal_city["Cloudiness"] == 0)]

#drop any null rows
ideal_city = ideal_city.dropna(axis=0, how="any")

#print dataframe
ideal_city

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
36,umm lajj,SA,25.02,37.27,78.75,47,0,1.34,1603671276
69,maceio,BR,-9.67,-35.74,77.00,88,0,6.93,1603671140
129,segou,ML,13.43,-6.22,79.25,17,0,3.38,1603671309
211,cayenne,GF,4.93,-52.33,78.80,88,0,2.24,1603671317
221,salalah,OM,17.02,54.09,75.20,60,0,5.82,1603671353
245,doka,SD,13.52,35.77,75.38,56,0,4.05,1603671362
297,mecca,SA,21.43,39.83,76.57,68,0,1.86,1603671382
322,kidal,ML,18.44,1.41,75.90,16,0,8.55,1603671391


### Hotel Map

In [6]:
#create new dataframe with only columns we're interested in
hotel_df = ideal_city[["City", "Country", "Latitude", "Longitude"]]

#create a new empty column for hotel name
hotel_df["Hotel Name"] = ""

#print dataframe
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
36,umm lajj,SA,25.02,37.27,
69,maceio,BR,-9.67,-35.74,
129,segou,ML,13.43,-6.22,
211,cayenne,GF,4.93,-52.33,
221,salalah,OM,17.02,54.09,
245,doka,SD,13.52,35.77,
297,mecca,SA,21.43,39.83,
322,kidal,ML,18.44,1.41,


In [7]:
#create a function to find nearby locations
def findnearby(params):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    data=response.json()
    return data

In [8]:
#set parameters for findnearby function
params = {"key" : g_key, "type" : "lodging", "radius" : 5000}

#for loop to iterate through each row of dataframe
for index, row in hotel_df.iterrows():
    
    #get latitude and longitude from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    #create location as new parameter for function using lat and lng
    params["location"] = f"{lat}, {lng}"
    
    #generate data using findnearby function with above parameters
    data = findnearby(params)
    
    #add hotel name to dataframe if there is a hotel nearby
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
    
    #if there is no hotel nearby, show which rows are missing
    except:
        print(f"missing value for row number {index}")

missing value for row number 245
missing value for row number 322


In [9]:
#show final dataframe with hotel names
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
36,umm lajj,SA,25.02,37.27,Moon light Furnished Units
69,maceio,BR,-9.67,-35.74,Mercure Maceio Pajucara Hotel
129,segou,ML,13.43,-6.22,Comme chez soi
211,cayenne,GF,4.93,-52.33,Hôtel Le Dronmi
221,salalah,OM,17.02,54.09,HAMDAN PLAZA HOTEL
245,doka,SD,13.52,35.77,
297,mecca,SA,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
322,kidal,ML,18.44,1.41,


In [10]:
#use the template to add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

#store the datafram row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
#add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))